In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
import sys

# TODO: change this to the path to your homework folder
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = '487/Project/'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))
os.chdir(GOOGLE_DRIVE_PATH)

Mounted at /content/drive
['Data']


In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [7]:
model = models.vgg19(pretrained=True)
# model.features = nn.Sequential(*list(model.features.children())[:-1])
model.classifier = nn.Sequential(
    nn.Linear(in_features=25088, out_features=4096, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=4096, out_features=200, bias=True)
)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
# Define a function to extract embeddings from an image
def extract_embedding(img_path):
    # Load image and preprocess
    img = Image.open(img_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    img = preprocess(img)

    # Extract features from the image using VGG19 model
    with torch.no_grad():
        features = model(img.unsqueeze(0)).squeeze()

    # Reshape features to a 200*1 embedding
    embedding = features.view(-1, 1)

    return embedding

In [9]:
embeddings = ()
path = '/Data/train_images'
for i in range(1,7001):
  path = path + i + '.jpg'
  embeddings = embeddings + extract_embedding(path)

embeddings = torch.stack(embeddings)
print(embeddings.shape)
torch.save(embeddings, 'VGG_basic.pt')

torch.Size([200, 1])
